# 2.1 Download data

In [1]:
import os
import requests

# location of the pipeline metadata store
_pipeline_root = './pipeline/'
## download the dataset
# Directory of the raw data files 
_data_root = './data/covertype'
# Path to the raw training data
_data_filepath = os.path.join(_data_root, 'covertype_train.csv')
# Download data
os.makedirs(_data_root, exist_ok=True)
#if not os.path.isfile(_data_filepath):
#https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/
url = 'https://docs.google.com/uc?export= \
download&confirm={{VALUE}}&id=1lVF1BCWLH4eXXV_YOJzjR7xZjj-wAGj9'
r = requests.get(url, allow_redirects=True, stream=True)
open(_data_filepath, 'wb').write(r.content)


6405459

## Preview Data

In [2]:
!head {_data_filepath}

Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
2991,119,7,67,11,1015,233,234,133,1570,Commanche,C7202,1
2876,3,18,485,71,2495,192,202,144,1557,Commanche,C7757,1
3171,315,2,277,9,4374,213,237,162,1052,Rawah,C7745,0
3087,342,13,190,31,4774,193,221,166,752,Rawah,C7745,0
2835,158,10,212,41,3596,231,242,141,3280,Rawah,C4744,1
3185,45,7,470,68,883,222,225,138,2187,Commanche,C7201,0
3157,88,29,361,95,1756,248,179,39,2766,Rawah,C7745,1
3152,310,7,0,0,5549,201,235,174,2100,Rawah,C7202,1
3302,119,23,309,-30,806,252,212,74,3537,Neota,C7756,1


# 3. Select Features

## Sub-Set number variables

In [3]:
import pandas as pd
# load dataset
df_total = pd.read_csv(_data_filepath)
# filter number columns
df_numeric = df_total.select_dtypes(include=['number'])
print(df_numeric.head())
print(f"{df_total.shape} {df_numeric.shape}")



   Elevation  Aspect  Slope  Horizontal_Distance_To_Hydrology  \
0       2991     119      7                                67   
1       2876       3     18                               485   
2       3171     315      2                               277   
3       3087     342     13                               190   
4       2835     158     10                               212   

   Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways  \
0                              11                             1015   
1                              71                             2495   
2                               9                             4374   
3                              31                             4774   
4                              41                             3596   

   Hillshade_9am  Hillshade_Noon  Hillshade_3pm  \
0            233             234            133   
1            192             202            144   
2            213             237   

## Prepapre input features and destination variale

In [4]:
X = df_numeric.drop(columns=['Cover_Type']) 
y = df_numeric['Cover_Type']

## Select features

In [5]:
from sklearn.feature_selection import SelectKBest, f_classif

# Create a SelectKBest object
selector = SelectKBest(k=8)

# Apply feature selection to the input data
X_new = selector.fit_transform(X, y)

# Get the scores of the selected features
scores = selector.scores_

# Get the indices of the selected features
selected_features_indices = selector.get_support(indices=True)

# Get the selected features
selected_features = X.iloc[:, selected_features_indices]
selected_features_df = pd.DataFrame(selected_features, columns=df_numeric.columns[selected_features_indices])
selected_features_df['Cover_Type'] = y
selected_features_df.to_csv(_data_filepath, index=False)

print(selected_features)



        Elevation  Slope  Horizontal_Distance_To_Hydrology  \
0            2991      7                                67   
1            2876     18                               485   
2            3171      2                               277   
3            3087     13                               190   
4            2835     10                               212   
...           ...    ...                               ...   
116198       3150     16                               285   
116199       3125     13                               234   
116200       3166     11                                67   
116201       3154     14                               738   
116202       3031      4                               426   

        Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways  \
0                                   11                             1015   
1                                   71                             2495   
2                             

## 4.1 Data Pipeline - Interactive context

In [6]:
import tensorflow as tf

from tfx.components import CsvExampleGen
from tfx.components import ExampleValidator
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Transform

from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict

import os
import pprint
pp = pprint.PrettyPrinter()

2024-03-04 05:07:37.341871: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-04 05:07:38.496034: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-03-04 05:07:38.496197: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such 

In [7]:
# Initialize the InteractiveContext with a local sqlite file.
# If you leave `_pipeline_root` blank, then the db will be created in a temporary directory.
# You can safely ignore the warning about the missing config file.
context = InteractiveContext(pipeline_root=_pipeline_root)

# Run TFX components interactively

With that, you can now run the pipeline interactively. You will see how to do that as you go through the different components below.

## ExampleGen

You will start the pipeline with the ExampleGen component. This will:

- split the data into training and evaluation sets (by default: 2/3 train, 1/3 eval).
- convert each data row into tf.train.Example format. This protocol buffer is designed for Tensorflow operations and is used by the TFX components.
- compress and save the data collection under the _pipeline_root directory for other components to access. These examples are stored in TFRecord format. This optimizes read and write operations within Tensorflow especially if you have a large collection of data.

# 4.2 Data Pipeline - Generating examples

In [8]:
# Instantiate ExampleGen with the input CSV dataset
example_gen = CsvExampleGen(input_base=_data_root)

In [9]:
# Execute the component
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

You will notice that an output cell showing the execution results is automatically shown. This metadata is recorded into the database created earlier. This allows you to keep track of your project runs. For example, if you run it again, you will notice the .execution_id incrementing.

In [10]:
# get the artifact object
artifact = example_gen.outputs['examples'].get()[0]

# print split names and uri
print(f'split names: {artifact.split_names}')
print(f'artifact uri: {artifact.uri}')

split names: ["train", "eval"]
artifact uri: ./pipeline/CsvExampleGen/examples/1


If you're wondering , the number in ./pipeline/CsvExampleGen/examples/{number} is the execution id associated with that dataset. If you restart the kernel of this workspace and re-run up to this cell, you will notice a new folder with a different id name created. This shows that TFX is keeping versions of your data so you can roll back if you want to investigate a particular execution.
As mentioned, the ingested data is stored in the directory shown in the uri field. It is also compressed using gzip and you can verify by running the cell below.

In [11]:
# Get the URI of the output artifact representing the training examples
train_uri = os.path.join(artifact.uri, 'Split-train')

# See the contents of the `train` folder
!ls {train_uri}

data_tfrecord-00000-of-00001.gz


In a notebook environment, it may be useful to examine a few examples of the data especially if you're still experimenting. Since the data collection is saved in TFRecord format, you will need to use methods that work with that data type. You will need to unpack the individual examples from the TFRecord file and format it for printing. Let's do that in the following cells:

In [12]:
# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

2024-03-04 05:08:18.638010: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-03-04 05:08:18.638081: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-03-04 05:08:18.638105: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (69316d1f3ff0): /proc/driver/nvidia/version does not exist
2024-03-04 05:08:18.638649: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical o

In [13]:
# Define a helper function to get individual examples
def get_records(dataset, num_records):
    '''Extracts records from the given dataset.
    Args:
        dataset (TFRecordDataset): dataset saved by ExampleGen
        num_records (int): number of records to preview
    '''
    
    # initialize an empty list
    records = []
    
    # Use the `take()` method to specify how many records to get
    for tfrecord in dataset.take(num_records):
        
        # Get the numpy property of the tensor
        serialized_example = tfrecord.numpy()
        
        # Initialize a `tf.train.Example()` to read the serialized data
        example = tf.train.Example()
        
        # Read the example data (output is a protocol buffer message)
        example.ParseFromString(serialized_example)
        
        # convert the protocol bufffer message to a Python dictionary
        example_dict = (MessageToDict(example))
        
        # append to the records list
        records.append(example_dict)
        
    return records

In [14]:
# Get 3 records from the dataset
sample_records = get_records(dataset, 3)

# Print the output
pp.pprint(sample_records)

[{'features': {'feature': {'Cover_Type': {'int64List': {'value': ['1']}},
                           'Elevation': {'int64List': {'value': ['2991']}},
                           'Hillshade_9am': {'int64List': {'value': ['233']}},
                           'Hillshade_Noon': {'int64List': {'value': ['234']}},
                           'Horizontal_Distance_To_Fire_Points': {'int64List': {'value': ['1570']}},
                           'Horizontal_Distance_To_Hydrology': {'int64List': {'value': ['67']}},
                           'Horizontal_Distance_To_Roadways': {'int64List': {'value': ['1015']}},
                           'Slope': {'int64List': {'value': ['7']}},
                           'Vertical_Distance_To_Hydrology': {'int64List': {'value': ['11']}}}}},
 {'features': {'feature': {'Cover_Type': {'int64List': {'value': ['1']}},
                           'Elevation': {'int64List': {'value': ['2876']}},
                           'Hillshade_9am': {'int64List': {'value': ['192']}},

# 4.3 StatisticsGen

The **StatisticsGen** component computes statistics over your dataset for data analysis, this component uses **TFDV** under the hood so its output will be familiar to you.

**StatisticsGen** takes as input the dataset we just ingested using **CsvExampleGen**.

In [15]:
# Instantiate StatisticsGen with the ExampleGen ingested dataset
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])

# Execute the component
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [16]:
# Show the output statistics
context.show(statistics_gen.outputs['statistics'])

# 4.4 SchemaGen

The **SchemaGen** component also uses **TFDV** to generate a schema based on your data statistics. 

SchemaGen will take as input the statistics that we generated with StatisticsGen, looking at the training split by default.

In [17]:
# Instantiate SchemaGen with the StatisticsGen ingested dataset
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    )

# Run the component
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [18]:
# Visualize the schema
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,-
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,-
'Hillshade_Noon',INT,required,,-
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,-
'Vertical_Distance_To_Hydrology',INT,required,,-


# 4.5 Schema curation

In [19]:
from tensorflow_metadata.proto.v0 import schema_pb2

In [20]:
schema_gen.outputs['schema']._artifacts[0]

.type,<class 'tfx.types.standard_artifacts.Schema'>
.uri,./pipeline/SchemaGen/schema/3


In [21]:
# Get the actual schema from folder
import tensorflow_data_validation as tfdv
schema = tfdv.load_schema_text(f"{schema_gen.outputs['schema']._artifacts[0].uri}/schema.pbtxt")

# Update the value ranges for Hillshade 9am, Hillshade Noon, and Slope
tfdv.set_domain(schema, 'Hillshade_9am', schema_pb2.IntDomain(min=0, max=255))
tfdv.set_domain(schema, 'Hillshade_Noon', schema_pb2.IntDomain(min=0, max=255))
tfdv.set_domain(schema, 'Slope', schema_pb2.IntDomain(min=0, max=90))

# Declare Cover Type as a categorical variable (swing)
tfdv.set_domain(schema, 'Cover_Type', schema_pb2.StringDomain(
    value=['0', '1', '2', '3', '4', '5', '6']
))

# Show the updated schema
context.show(schema)
print(type(schema))
print(type(schema_gen.outputs['schema']))
tfdv.write_schema_text(
    schema=schema, 
    output_path=f"{schema_gen.outputs['schema']._artifacts[0].uri}/schema.pbtxt"
)

feature {
  name: "Cover_Type"
  type: INT
  string_domain {
    value: "0"
    value: "1"
    value: "2"
    value: "3"
    value: "4"
    value: "5"
    value: "6"
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "Elevation"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "Hillshade_9am"
  type: INT
  int_domain {
    min: 0
    max: 255
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "Hillshade_Noon"
  type: INT
  int_domain {
    min: 0
    max: 255
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "Horizontal_Distance_To_Fire_Points"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "Horizontal_Di

<class 'tensorflow_metadata.proto.v0.schema_pb2.Schema'>
<class 'tfx.types.channel.OutputChannel'>


In [22]:
# Visualize the schema
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Cover_Type',INT,required,,'Cover_Type_domain'
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90
'Vertical_Distance_To_Hydrology',INT,required,,-


,Values
Domain,
'Cover_Type_domain',"'0', '1', '2', '3', '4', '5', '6'"


# 4.6 Enviromets

In [23]:
# Generate statistics from training dataset
eval_stats_train_set = tfdv.generate_statistics_from_csv(data_location=_data_filepath)
# Simulate service dataset by removing the label column
sampled_service_dataset = selected_features_df.drop(columns=['Cover_Type']).sample(n=500, random_state=42).reset_index(drop=True)
# Generate statistics from service dataset
eval_stats_service = tfdv.generate_statistics_from_dataframe(sampled_service_dataset)
# Compare evaluation data with training data
tfdv.visualize_statistics(lhs_statistics=eval_stats_service, rhs_statistics=eval_stats_train_set,
                          lhs_name='EVAL_DATASET', rhs_name='TRAIN_DATASET')

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [24]:
anomalies = tfdv.validate_statistics(statistics=eval_stats_service, schema=schema)
tfdv.display_anomalies(anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'Cover_Type',Column dropped,Column is completely missing


In [25]:
# All features are by default in both TRAINING and SERVING environments.
schema.default_environment.append('TRAINING')
schema.default_environment.append('SERVING')
# Specify that 'tips' feature is not in SERVING environment.
tfdv.get_feature(schema, 'Cover_Type').not_in_environment.append('SERVING')
serving_anomalies_with_env = tfdv.validate_statistics(
    eval_stats_service, schema, environment='SERVING')

tfdv.display_anomalies(serving_anomalies_with_env)


In [26]:
# Save Schema
tfdv.write_schema_text(
    schema=schema, 
    output_path=f"{schema_gen.outputs['schema']._artifacts[0].uri}/schema.pbtxt"
)
print(schema)

feature {
  name: "Cover_Type"
  type: INT
  string_domain {
    value: "0"
    value: "1"
    value: "2"
    value: "3"
    value: "4"
    value: "5"
    value: "6"
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  not_in_environment: "SERVING"
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "Elevation"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "Hillshade_9am"
  type: INT
  int_domain {
    min: 0
    max: 255
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "Hillshade_Noon"
  type: INT
  int_domain {
    min: 0
    max: 255
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "Horizontal_Distance_To_Fire_Points"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}


# 4.7 Generate new statistics with schema

In [27]:
from tensorflow_data_validation.statistics import stats_options
stats_options = stats_options.StatsOptions(schema=schema)
statistics_gen = tfdv.generate_statistics_from_dataframe(dataframe=sampled_service_dataset, stats_options=stats_options)
tfdv.visualize_statistics(statistics_gen)

# 4.8 Validate Anomalies

In [28]:
anomalies = tfdv.validate_statistics(statistics_gen, schema)
tfdv.display_anomalies(serving_anomalies_with_env)

# 4.9 Features Engineering

In [29]:
_cover_type_module_file = 'cover_type_constants.py'

In [30]:
%%writefile {_cover_type_module_file}
 
NUMERIC_FEATURE_KEYS = ['Elevation', 'Horizontal_Distance_To_Fire_Points', 'Horizontal_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways', 'Vertical_Distance_To_Hydrology']
 
FEATURE_RANGE_0_255 = ['Hillshade_9am', 'Hillshade_Noon']
 
FEATURE_RANGE_0_90 = ['Slope']
 
LABEL_KEY= 'Cover_Type'
 
def transformed_name(key):
    return key + '_xf'

Writing cover_type_constants.py


# 4.10 Preprocessing function

In [31]:
_transform_cover_type = 'cover_type_transform.py'

In [32]:
%%writefile {_transform_cover_type}
import tensorflow as tf
import tensorflow_transform as tft
import cover_type_constants

# Unpack the contents of the constants module

_NUMERIC_FEATURE_KEYS = cover_type_constants.NUMERIC_FEATURE_KEYS

_FEATURE_RANGE_0_255 = cover_type_constants.FEATURE_RANGE_0_255

_FEATURE_RANGE_0_90 = cover_type_constants.FEATURE_RANGE_0_90

_LABEL_KEY = cover_type_constants.LABEL_KEY

_transformed_name = cover_type_constants.transformed_name 
# Define the transformations
def preprocessing_fn(inputs):
    outputs = {}
 
    # Scale these features to the range [0,1]
    for key in _NUMERIC_FEATURE_KEYS:
        outputs[_transformed_name(key)] = tft.scale_to_0_1(
            inputs[key])
   
    # Bucketize these features
    for key in _FEATURE_RANGE_0_255:
        outputs[_transformed_name(key)] = tft.scale_by_min_max(
            inputs[key], 0.0, 255.0)
 
    # Convert strings to indices in a vocabulary
    for key in _FEATURE_RANGE_0_90:
        outputs[_transformed_name(key)] = tft.scale_by_min_max(
            inputs[key], 0.0, 90.0)
 
    # Convert the label strings to an index
    outputs[_transformed_name(_LABEL_KEY)] = (inputs[_LABEL_KEY])
 
    return outputs

Writing cover_type_transform.py


# 4.11 Transform

In [33]:
# Ignore TF warning messages
tf.get_logger().setLevel('ERROR')
 
# Instantiate the Transform component
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_transform_cover_type))
 
# Run the component
context.run(transform) 

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying cover_type_constants.py -> build/lib
copying preprocessing.py -> build/lib
copying cover_type_transform.py -> build/lib
installing to /tmp/tmp0i7vvrb5
running install
running install_lib
copying build/lib/cover_type_transform.py -> /tmp/tmp0i7vvrb5
copying build/lib/cover_type_constants.py -> /tmp/tmp0i7vvrb5
copying build/lib/preprocessing.py -> /tmp/tmp0i7vvrb5
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to 

/opt/conda/lib/python3.7/site-packages/setuptools/command/install.py:37: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  setuptools.SetuptoolsDeprecationWarning,


Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+c72ba397635cbf4faf08aa65f6d81c576e15814dddf88cf930ae96e350130494-py3-none-any.whl


Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+c72ba397635cbf4faf08aa65f6d81c576e15814dddf88cf930ae96e350130494-py3-none-any.whl


Processing ./pipeline/_wheels/tfx_user_code_Transform-0.0+c72ba397635cbf4faf08aa65f6d81c576e15814dddf88cf930ae96e350130494-py3-none-any.whl


ExecutionResult(
    component_id: Transform
    execution_id: 4
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [34]:
# Get the uri of the transform graph
transform_graph_uri = transform.outputs['transform_graph'].get()[0].uri
 
# List the subdirectories under the uri
os.listdir(transform_graph_uri)

['transformed_metadata', 'transform_fn', 'metadata']

In [35]:
# Get the URI of the output artifact representing the transformed examples
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')
 
# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]
 
# Create a `TFRecordDataset` to read these files
transformed_dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [37]:
# Get 3 records from the dataset
sample_records_xf = get_records(transformed_dataset, 3)
 
# Print the output
pp.pprint(sample_records_xf)

[{'features': {'feature': {'Cover_Type_xf': {'int64List': {'value': ['1']}},
                           'Elevation_xf': {'floatList': {'value': [0.5686275]}},
                           'Hillshade_9am_xf': {'floatList': {'value': [233.91731]}},
                           'Hillshade_Noon_xf': {'floatList': {'value': [234.92125]}},
                           'Horizontal_Distance_To_Fire_Points_xf': {'floatList': {'value': [0.21902902]}},
                           'Horizontal_Distance_To_Hydrology_xf': {'floatList': {'value': [0.04886944]}},
                           'Horizontal_Distance_To_Roadways_xf': {'floatList': {'value': [0.14263631]}},
                           'Slope_xf': {'floatList': {'value': [9.545455]}},
                           'Vertical_Distance_To_Hydrology_xf': {'floatList': {'value': [0.2214854]}}}}},
 {'features': {'feature': {'Cover_Type_xf': {'int64List': {'value': ['1']}},
                           'Elevation_xf': {'floatList': {'value': [0.5108095]}},
       

# 5 Metadata ML

In [38]:
_util = 'util.py'

In [39]:
%%writefile {_util}
 
import tensorflow as tf
import pandas as pd
from google.protobuf.json_format import MessageToDict
 
def get_records(dataset, num_records):
    '''Extracts records from the given dataset.
    Args:
        dataset (TFRecordDataset): dataset saved by ExampleGen
        num_records (int): number of records to preview
    '''
    # initialize an empty list
    records = []
    # Use the `take()` method to specify how many records to get
    for tfrecord in dataset.take(num_records):
        # Get the numpy property of the tensor
        serialized_example = tfrecord.numpy()
        # Initialize a `tf.train.Example()` to read the serialized data
        example = tf.train.Example()
        # Read the example data (output is a protocol buffer message)
        example.ParseFromString(serialized_example)
        # convert the protocol bufffer message to a Python dictionary
        example_dict = (MessageToDict(example))
        # append to the records list
        records.append(example_dict)
    return records
 
def display_types(types):
    # Helper function to render dataframes for the artifact and execution types
    table = {'id': [], 'name': []}
    for a_type in types:
        table['id'].append(a_type.id)
        table['name'].append(a_type.name)
    return pd.DataFrame(data=table)
 
def display_artifacts(store, artifacts, base_dir):
    # Helper function to render dataframes for the input artifacts
    table = {'artifact id': [], 'type': [], 'uri': []}
    for a in artifacts:
        table['artifact id'].append(a.id)
        artifact_type = store.get_artifact_types_by_id([a.type_id])[0]
        table['type'].append(artifact_type.name)
        table['uri'].append(a.uri.replace(base_dir, './'))
    return pd.DataFrame(data=table)
 
def display_properties(store, node):
    # Helper function to render dataframes for artifact and execution properties
    table = {'property': [], 'value': []}
    for k, v in node.properties.items():
        table['property'].append(k)
        table['value'].append(
            v.string_value if v.HasField('string_value') else v.int_value)
    for k, v in node.custom_properties.items():
        table['property'].append(k)
        table['value'].append(
            v.string_value if v.HasField('string_value') else v.int_value)
    return pd.DataFrame(data=table)

Writing util.py


In [40]:
import util as u_t
import ml_metadata as mlmd
from ml_metadata.proto import metadata_store_pb2
from ml_metadata.metadata_store import metadata_store
 
connection_config = metadata_store_pb2.ConnectionConfig()
connection_config.sqlite.filename_uri = 'pipeline/metadata.sqlite'
connection_config.sqlite.connection_mode = 3
store = metadata_store.MetadataStore(connection_config)

In [41]:
artifact_types = store.get_artifact_types()

In [42]:
u_t.display_types(artifact_types)

,id,name
0,14,Examples
1,16,ExampleStatistics
2,18,Schema
3,20,TransformGraph
4,21,TransformCache
5,22,ExampleAnomalies
